In [2]:
import numpy as np
from neuromaps import datasets, images, parcellate


def get_lr_intersections(gifti, gifti_relabeled):
    """get the indices of regions that cross both L-R hemispheres."""
    hemi_intersection = np.intersect1d(gifti[0].agg_data(), gifti[1].agg_data())
    l_mask = np.isin(gifti[0].agg_data(), hemi_intersection)
    r_mask = np.isin(gifti[1].agg_data(), hemi_intersection)
    l_idx = np.unique(gifti_relabeled[0].agg_data()[l_mask])
    r_idx = np.unique(gifti_relabeled[1].agg_data()[r_mask])
    return l_idx[1:] - 1, r_idx[1:] - 1  # ?? ignore 0 label and reindex to start at 0 ??


# cifti ICA parcellation file
dlabel = "./data/nregions-300_cifti.dlabel.nii"

# cifti to surface removes volumetric data: 300 -> 106 regions
gifti = images.dlabel_to_gifti(dlabel)

# make label IDs consecutive across hemispheres: 106 -> 178 regions
gifti_relabeled = images.relabel_gifti(gifti)

# get intersections of gifti_relabeled: 72 regions
l_idx, r_idx = get_lr_intersections(gifti, gifti_relabeled)

# initialize parcellator
parc = parcellate.Parcellater(gifti_relabeled, "fsLR")

# extract T1w/T2w ratio defined by ICA parcellation
t1t2_map = datasets.fetch_annotation(source="hcps1200", desc="myelinmap", space="fsLR", den="32k")
t1t2_lr_split = parc.fit_transform(t1t2_map, "fsLR")  # (178, ): 88L <-> 90R

# average regions that cross both L-R hemispheres
t1t2_lr_avg = np.zeros(106)
t1t2_lr_avg[l_idx] = (t1t2_lr_split[l_idx] + t1t2_lr_split[r_idx]) / 2
zero_idx = np.where(t1t2_lr_avg == 0)[0]
t1t2_lr_avg[zero_idx] = t1t2_lr_split[zero_idx]
